# Claims Underwriting Data Generator
## Finance Analytics for Chief Underwriting Officer

This notebook generates realistic insurance data for:
- Claims risk assessment
- Underwriting optimization
- Financial performance analysis

In [0]:
%pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 13.5 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random
import uuid

# Initialize Faker
fake = Faker()
fake.seed_instance(42)  # For reproducible results
np.random.seed(42)
random.seed(42)

# Configuration
NUM_CUSTOMERS = 10000
NUM_POLICIES = 25000
NUM_CLAIMS = 50000

## Generate Customers Data

In [0]:
def generate_customers():
    """Generate customers table with realistic data distributions"""
    print("Generating customers data...")
    
    customers = []
    
    for i in range(NUM_CUSTOMERS):
        customer_id = f"CUST_{str(i+1).zfill(6)}"
        
        # Business logic for customer attributes
        customer_type = np.random.choice(['individual', 'business'], p=[0.85, 0.15])
        
        if customer_type == 'individual':
            age = np.random.normal(45, 15)
            age = max(18, min(85, int(age)))  # Clamp between 18-85
            annual_income = np.random.lognormal(10.5, 0.5)  # Realistic income distribution
            employment_status = np.random.choice(
                ['employed', 'self_employed', 'retired', 'unemployed'],
                p=[0.65, 0.15, 0.15, 0.05]
            )
            marital_status = np.random.choice(
                ['single', 'married', 'divorced', 'widowed'],
                p=[0.3, 0.5, 0.15, 0.05]
            )
        else:  # business
            age = None  # Business entities don't have age
            annual_income = np.random.lognormal(12, 0.8)  # Higher business revenues
            employment_status = 'business_entity'
            marital_status = None
        
        # Credit score correlation with income and age
        if customer_type == 'individual':
            base_credit = 650 + (annual_income / 100000) * 50 + (age - 25) * 2
            credit_score = int(np.random.normal(base_credit, 80))
            credit_score = max(300, min(850, credit_score))
        else:
            credit_score = int(np.random.normal(700, 60))  # Business credit scores
            credit_score = max(400, min(850, credit_score))
        
        # Years with company (loyalty correlation)
        years_with_company = np.random.exponential(3.5)
        years_with_company = min(25, round(years_with_company, 1))
        
        # Geographic risk zone
        geographic_risk_zone = np.random.choice(
            ['low', 'medium', 'high'],
            p=[0.4, 0.45, 0.15]
        )
        
        # Prior claims count (correlated with risk zone)
        risk_multiplier = {'low': 0.5, 'medium': 1.0, 'high': 2.0}[geographic_risk_zone]
        prior_claims_count = np.random.poisson(1.2 * risk_multiplier)
        prior_claims_count = min(15, prior_claims_count)
        
        # Customer loyalty tier (based on years and income)
        if years_with_company < 1:
            loyalty_tier = 'bronze'
        elif years_with_company < 3:
            loyalty_tier = 'silver'
        elif years_with_company < 7 or annual_income > 75000:
            loyalty_tier = 'gold'
        else:
            loyalty_tier = 'platinum'
        
        created_date = fake.date_time_between(
            start_date='-10y', 
            end_date=datetime.now() - timedelta(days=30)
        )
        
        customers.append({
            'customer_id': customer_id,
            'customer_type': customer_type,
            'age': age,
            'credit_score': credit_score,
            'years_with_company': years_with_company,
            'annual_income': round(annual_income, 2),
            'employment_status': employment_status,
            'marital_status': marital_status,
            'geographic_risk_zone': geographic_risk_zone,
            'prior_claims_count': prior_claims_count,
            'customer_loyalty_tier': loyalty_tier,
            'created_date': created_date
        })
    
    return pd.DataFrame(customers)

## Generate Policies Data

In [0]:
def generate_policies(customers_df):
    """Generate policies table with proper foreign key relationships"""
    print("Generating policies data...")
    
    policies = []
    
    # Create policy distribution - some customers have multiple policies
    customer_policy_counts = np.random.choice([1, 2, 3, 4], size=len(customers_df), p=[0.4, 0.3, 0.2, 0.1])
    
    policy_counter = 1
    
    for idx, row in customers_df.iterrows():
        customer_id = row['customer_id']
        num_policies = customer_policy_counts[idx]
        
        for policy_num in range(num_policies):
            if policy_counter > NUM_POLICIES:
                break
                
            policy_id = f"POL_{str(policy_counter).zfill(7)}"
            
            # Policy type distribution
            if row['customer_type'] == 'business':
                policy_type = np.random.choice(
                    ['commercial', 'auto', 'liability'],
                    p=[0.5, 0.3, 0.2]
                )
            else:
                policy_type = np.random.choice(
                    ['auto', 'home', 'life', 'health'],
                    p=[0.4, 0.3, 0.2, 0.1]
                )
            
            # Coverage amount based on policy type and customer income
            coverage_multipliers = {
                'auto': (15000, 100000),
                'home': (100000, 1000000),
                'life': (50000, 500000),
                'health': (50000, 250000),
                'commercial': (100000, 5000000),
                'liability': (500000, 10000000)
            }
            
            min_cov, max_cov = coverage_multipliers[policy_type]
            income_factor = min(2.0, row['annual_income'] / 50000)
            coverage_amount = np.random.uniform(min_cov, max_cov * income_factor)
            
            # Deductible (inverse relationship with coverage)
            deductible_options = [500, 1000, 2500, 5000, 10000]
            # Higher coverage tends to have higher deductibles
            deductible_weights = [0.1, 0.2, 0.3, 0.3, 0.1] if coverage_amount > 500000 else [0.4, 0.3, 0.2, 0.08, 0.02]
            deductible = np.random.choice(deductible_options, p=deductible_weights)
            
            # Premium calculation (simplified but realistic)
            base_premium = coverage_amount * 0.005  # 0.5% of coverage
            risk_multipliers = {'low': 0.8, 'medium': 1.0, 'high': 1.4}
            risk_multiplier = risk_multipliers[row['geographic_risk_zone']]
            
            # Credit score impact
            credit_multiplier = max(0.7, min(1.3, (850 - row['credit_score']) / 400))
            
            premium_amount = base_premium * risk_multiplier * credit_multiplier
            premium_amount = round(premium_amount, 2)
            
            # Policy dates
            policy_start_date = fake.date_between(
                start_date=row['created_date'].date(),
                end_date=datetime.now().date()
            )
            policy_end_date = policy_start_date + timedelta(days=365)
            
            # Policy status
            if policy_end_date < datetime.now().date():
                policy_status = np.random.choice(['expired', 'active'], p=[0.7, 0.3])
            else:
                policy_status = np.random.choice(['active', 'cancelled'], p=[0.9, 0.1])
            
            # Risk tier based on multiple factors
            risk_score = (
                (row['prior_claims_count'] * 0.3) +
                ({'low': 1, 'medium': 2, 'high': 3}[row['geographic_risk_zone']] * 0.4) +
                ((850 - row['credit_score']) / 200 * 0.3)
            )
            
            if risk_score < 1.5:
                risk_tier = 'low'
            elif risk_score < 2.5:
                risk_tier = 'medium'
            else:
                risk_tier = 'high'
            
            agent_id = f"AGT_{random.randint(1, 50):03d}"
            
            policies.append({
                'policy_id': policy_id,
                'customer_id': customer_id,
                'policy_type': policy_type,
                'coverage_amount': round(coverage_amount, 2),
                'deductible': deductible,
                'premium_amount': premium_amount,
                'policy_start_date': policy_start_date,
                'policy_end_date': policy_end_date,
                'policy_status': policy_status,
                'risk_tier': risk_tier,
                'agent_id': agent_id,
                'created_date': policy_start_date
            })
            
            policy_counter += 1
            
        if policy_counter > NUM_POLICIES:
            break
    
    return pd.DataFrame(policies)

## Generate Claims Data

In [0]:
def generate_claims(policies_df, customers_df):
    """Generate claims table with realistic claim patterns"""
    print("Generating claims data...")
    
    claims = []
    
    # Create claims distribution - not all policies have claims
    policies_with_claims = policies_df.sample(n=min(NUM_CLAIMS // 2, len(policies_df)))
    
    claim_counter = 1
    
    for idx, policy_row in policies_with_claims.iterrows():
        # Some policies have multiple claims
        num_claims = np.random.choice([1, 2, 3], p=[0.7, 0.25, 0.05])
        
        # Get customer data for this policy
        customer_data = customers_df[customers_df['customer_id'] == policy_row['customer_id']].iloc[0]
        
        for claim_num in range(num_claims):
            if claim_counter > NUM_CLAIMS:
                break
                
            claim_id = f"CLM_{str(claim_counter).zfill(8)}"
            
            # Claim type based on policy type
            claim_type_mapping = {
                'auto': ['collision', 'theft', 'liability'],
                'home': ['fire', 'theft', 'flood', 'liability'],
                'life': ['death_benefit'],
                'health': ['medical', 'emergency'],
                'commercial': ['liability', 'fire', 'theft'],
                'liability': ['liability']
            }
            
            available_claim_types = claim_type_mapping.get(policy_row['policy_type'], ['other'])
            claim_type = np.random.choice(available_claim_types)
            
            # Claim date within policy period
            claim_date_filed = fake.date_between(
                start_date=policy_row['policy_start_date'],
                end_date=min(policy_row['policy_end_date'], datetime.now().date())
            )
            
            # Incident date before claim date
            incident_date = claim_date_filed - timedelta(days=random.randint(0, 30))
            
            # Claim amount based on policy coverage and claim type
            severity_multipliers = {
                'collision': 0.15, 'theft': 0.25, 'fire': 0.4, 'flood': 0.3,
                'medical': 0.1, 'emergency': 0.2, 'liability': 0.2,
                'death_benefit': 1.0, 'other': 0.15
            }
            
            severity = severity_multipliers.get(claim_type, 0.15)
            base_amount = policy_row['coverage_amount'] * severity
            
            # Add randomness
            claim_amount_requested = base_amount * np.random.uniform(0.5, 1.5)
            claim_amount_requested = round(max(100, claim_amount_requested), 2)
            
            # Claim complexity score (1-5 scale)
            complexity_factors = {
                'collision': 2.5, 'theft': 3.0, 'fire': 4.0, 'flood': 4.5,
                'medical': 2.0, 'emergency': 3.5, 'liability': 4.5,
                'death_benefit': 5.0, 'other': 2.0
            }
            base_complexity = complexity_factors.get(claim_type, 2.0)
            claim_complexity_score = round(np.random.normal(base_complexity, 0.5), 2)
            claim_complexity_score = max(1.0, min(5.0, claim_complexity_score))
            
            # Fraud risk score (higher for certain patterns)
            fraud_risk_base = 0.1
            
            # Risk factors
            if claim_amount_requested > policy_row['coverage_amount'] * 0.8:
                fraud_risk_base += 0.3  # High claim amount
            if customer_data['prior_claims_count'] > 3:
                fraud_risk_base += 0.2  # Frequent claimant
            if policy_row['risk_tier'] == 'high':
                fraud_risk_base += 0.2  # High risk customer
            if claim_type in ['theft', 'fire']:
                fraud_risk_base += 0.1  # Claim types with higher fraud rates
            
            fraud_risk_score = round(min(1.0, fraud_risk_base + np.random.uniform(-0.1, 0.1)), 2)
            
            # Claim status and settlement logic
            days_since_filed = (datetime.now().date() - claim_date_filed).days
            
            if days_since_filed < 7:
                claim_status = 'submitted'
                settlement_amount = None
                days_to_settlement = None
            elif days_since_filed < 30:
                claim_status = np.random.choice(['under_review', 'submitted'], p=[0.8, 0.2])
                settlement_amount = None
                days_to_settlement = None
            else:
                # Approval probability based on fraud risk and other factors
                approval_prob = max(0.1, 0.9 - (fraud_risk_score * 0.6))
                
                if np.random.random() < approval_prob:
                    claim_status = np.random.choice(['approved', 'settled'], p=[0.3, 0.7])
                    # Settlement typically less than requested
                    settlement_amount = claim_amount_requested * np.random.uniform(0.7, 1.0)
                    settlement_amount = round(max(0, settlement_amount), 2)
                else:
                    claim_status = 'denied'
                    settlement_amount = 0
                
                # Days to settlement based on complexity
                base_days = 30 + (claim_complexity_score - 1) * 15
                days_to_settlement = int(np.random.normal(base_days, 10))
                days_to_settlement = max(7, min(365, days_to_settlement))
            
            adjuster_id = f"ADJ_{random.randint(1, 25):03d}"
            
            description = f"{claim_type.replace('_', ' ').title()} claim for policy {policy_row['policy_id']}"
            
            claims.append({
                'claim_id': claim_id,
                'policy_id': policy_row['policy_id'],
                'claim_type': claim_type,
                'claim_amount_requested': claim_amount_requested,
                'claim_date_filed': claim_date_filed,
                'claim_status': claim_status,
                'settlement_amount': settlement_amount,
                'days_to_settlement': days_to_settlement,
                'adjuster_id': adjuster_id,
                'claim_complexity_score': claim_complexity_score,
                'fraud_risk_score': fraud_risk_score,
                'incident_date': incident_date,
                'description': description,
                'created_date': claim_date_filed
            })
            
            claim_counter += 1
            
        if claim_counter > NUM_CLAIMS:
            break
    
    return pd.DataFrame(claims)

## Execute Data Generation

In [0]:
# Main execution
print("Starting data generation for Claims Underwriting Analytics...")
print(f"Target: {NUM_CUSTOMERS:,} customers, {NUM_POLICIES:,} policies, {NUM_CLAIMS:,} claims")

# Generate datasets
customers_df = generate_customers()
print(f"✅ Generated {len(customers_df):,} customers")

policies_df = generate_policies(customers_df)
print(f"✅ Generated {len(policies_df):,} policies")

claims_df = generate_claims(policies_df, customers_df)
print(f"✅ Generated {len(claims_df):,} claims")

Starting data generation for Claims Underwriting Analytics...
Target: 10,000 customers, 25,000 policies, 50,000 claims
Generating customers data...
✅ Generated 10,000 customers
Generating policies data...
✅ Generated 20,109 policies
Generating claims data...
✅ Generated 27,065 claims


## Data Quality Summary

In [0]:
# Data quality checks
print("📊 Data Quality Summary:")
print(f"Customers with policies: {customers_df['customer_id'].isin(policies_df['customer_id']).sum():,}")
print(f"Policies with claims: {policies_df['policy_id'].isin(claims_df['policy_id']).sum():,}")
print(f"Average policies per customer: {len(policies_df) / len(customers_df):.2f}")
print(f"Claims rate: {len(claims_df) / len(policies_df):.2%}")

# Display sample data
print("\n🔍 Sample Data Preview:")
print("\nCustomers Sample:")
display(customers_df.head(3))
print("\nPolicies Sample:")
display(policies_df.head(3))
print("\nClaims Sample:")
display(claims_df.head(3))

📊 Data Quality Summary:
Customers with policies: 10,000
Policies with claims: 20,109
Average policies per customer: 2.01
Claims rate: 134.59%

🔍 Sample Data Preview:

Customers Sample:


customer_id,customer_type,age,credit_score,years_with_company,annual_income,employment_status,marital_status,geographic_risk_zone,prior_claims_count,customer_loyalty_tier,created_date
CUST_000001,individual,28.0,699,4.3,42593.25,employed,single,low,2,gold,2021-11-24T16:05:03.441Z
CUST_000002,individual,60.0,659,1.2,27290.94,employed,married,medium,0,silver,2015-10-22T00:13:41.119Z
CUST_000003,individual,34.0,850,0.2,12458.89,self_employed,single,medium,0,bronze,2018-04-14T16:14:30.927Z



Policies Sample:


policy_id,customer_id,policy_type,coverage_amount,deductible,premium_amount,policy_start_date,policy_end_date,policy_status,risk_tier,agent_id,created_date
POL_0000001,CUST_000001,home,406201.97,5000,1137.37,2025-05-13,2026-05-13,active,low,AGT_041,2025-05-13
POL_0000002,CUST_000002,life,69775.86,1000,244.22,2024-08-15,2025-08-15,active,low,AGT_008,2024-08-15
POL_0000003,CUST_000002,auto,48575.96,2500,170.02,2022-08-29,2023-08-29,expired,low,AGT_002,2022-08-29



Claims Sample:


claim_id,policy_id,claim_type,claim_amount_requested,claim_date_filed,claim_status,settlement_amount,days_to_settlement,adjuster_id,claim_complexity_score,fraud_risk_score,incident_date,description,created_date
CLM_00000001,POL_0011634,death_benefit,76470.29,2022-09-12,settled,54456.5,97.0,ADJ_014,4.92,0.11,2022-09-12,Death Benefit claim for policy POL_0011634,2022-09-12
CLM_00000002,POL_0001338,medical,29050.15,2025-07-19,submitted,null,null,ADJ_022,2.08,0.14,2025-06-24,Medical claim for policy POL_0001338,2025-07-19
CLM_00000003,POL_0008643,liability,15473.58,2022-04-19,settled,13312.97,87.0,ADJ_006,4.68,0.15,2022-03-22,Liability claim for policy POL_0008643,2022-04-19


## Save to Unity Catalog

Ready to save these DataFrames to Delta tables in Unity Catalog!

In [0]:
print("🎉 Data generation complete! DataFrames are ready for Unity Catalog.")
print("\nNext steps:")
print("1. Configure Unity Catalog catalog and schema")
print("2. Save DataFrames as Delta tables")
print("3. Set up Delta Live Tables pipeline")
print("4. Build ML model for claims prediction")

🎉 Data generation complete! DataFrames are ready for Unity Catalog.

Next steps:
1. Configure Unity Catalog catalog and schema
2. Save DataFrames as Delta tables
3. Set up Delta Live Tables pipeline
4. Build ML model for claims prediction
